# Baseline solution for seismic facies identification challenge

### Architecture: U-Net
### Language: Tensorflow/keras

In [26]:
import tensorflow as tf
import keras as K

from keras.layers import Input, BatchNormalization, Dropout, Dense, Activation
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate

import numpy as np
from tqdm import tqdm
import cv2


### Define Network architecture

In [3]:
# defines convolutional block

def conv2d_block(input_tensor,n_filters, kernel_size=3, batchnorm=True):
    x = Conv2D(n_filters, (kernel_size,kernel_size),padding='same',\
               kernel_initializer='he_normal')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv2D(n_filters, (kernel_size, kernel_size), padding='same',\
               kernel_initializer='he_normal')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x

# defines unet architecture

def unet(img_input, out_channel, n_filters=64, dropout=0.1, k_size=3, batchnorm = True):
    c1 = conv2d_block(img_input, n_filters*1, kernel_size=k_size, batchnorm=batchnorm)
    p1 = MaxPooling2D((2,2))(c1)
    
    c2 = conv2d_block(p1, n_filters*2, kernel_size=k_size, batchnorm=batchnorm)
    p2 = MaxPooling2D((2,2))(c2)
    
    c3 = conv2d_block(p2, n_filters*4, kernel_size=k_size, batchnorm=batchnorm)
    p3 = MaxPooling2D((2,2))(c3)
    
    c4 = conv2d_block(p3, n_filters*8, kernel_size=k_size, batchnorm=batchnorm)
    p4 = MaxPooling2D((2,2))(c4)
    
    c5 = conv2d_block(p4, n_filters*16, batchnorm=batchnorm)
    
    
    
    u6 = Conv2DTranspose(n_filters*8, (k_size,k_size), strides = (2,2), padding='same')(c5)
    u6 = concatenate([u6,c4])
    c6 = conv2d_block(u6, n_filters*8, kernel_size=k_size, batchnorm=batchnorm)
    
    u7 = Conv2DTranspose(n_filters*4, (k_size,k_size),strides = (2,2), padding='same')(c6)
    u7 = concatenate([u7,c3])
    c7 = conv2d_block(u7, n_filters*4, kernel_size=k_size, batchnorm=batchnorm)
    
    u8 = Conv2DTranspose(n_filters*2, (k_size,k_size), strides = (2,2), padding='same')(c7)
    u8 = concatenate([u8,c2])
    c8 = conv2d_block(u8, n_filters*2, kernel_size=k_size, batchnorm=batchnorm)
    
    u9 = Conv2DTranspose(n_filters*1, (k_size,k_size), strides = (2,2), padding='same')(c8)
    u9 = concatenate([u9,c1])
    c9 = conv2d_block(u9, n_filters*1, kernel_size=k_size, batchnorm=batchnorm)
    
    outputs = Conv2D(out_channel, (1,1), activation='softmax')(c9)
    model = Model(inputs= [img_input], outputs= [outputs], name ='U-Net')
    
    return model

## Data pre-processing

In [23]:
data_train = np.load('./dataset/data_train.npz')['data']
label_train = np.load('./dataset/labels_train.npz')['labels']


data_patch = np.concatenate((data_train[:,:,:295], data_train[:,:,295:]))
label_patch = np.concatenate((label_train[:,:,:295], label_train[:,:,295:]))

In [24]:
data_patch.shape

(2012, 782, 295)

In [28]:
training_image_data = []
training_label_data = []

for i in tqdm (range(0, 2012)):
    image = data_patch[i,:,:]
    label = label_patch[i,:,:]
    
    image = np.expand_dims(image,axis=2).astype('float32')
    label = np.expand_dims(label,axis=2).astype('float32')
    
    image = cv2.resize(image, (256,256))
    label = cv2.resize(label,(256,256))
    
    training_image_data.append(image)
    training_label_data.append(label)
    
    
# convert data to array
training_image_data = np.asarray(training_image_data)
training_label_data = np.asarray(training_label_data)
training_label_data = np.array(training_label_data, dtype=int)

100%|██████████| 2012/2012 [00:09<00:00, 205.84it/s]
